# BERT Pipeline - Fine-tuning для NLP задач

Этот notebook содержит готовые пайплайны для:
- Классификации текста (sentiment, topic classification)
- Named Entity Recognition (NER)
- Question Answering
- Token Classification
- Text Embeddings

## 1. Установка зависимостей

In [ ]:
!pip install transformers datasets accelerate torch scikit-learn -q

## 2. Импорты

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    pipeline
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 3. Загрузка данных
**Подставьте свои данные здесь**

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат для классификации текста:
# df должен содержать колонки 'text' и 'label'

# Вариант 1: Из CSV
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Вариант 2: Пример данных
# data = {
#     'text': ['Отличный фильм!', 'Ужасно скучно', 'Неплохо'],
#     'label': [1, 0, 1]  # 0 - негативный, 1 - позитивный
# }
# train_df = pd.DataFrame(data)

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nПример данных:\n{train_df.head()}")

## 4. Настройка задачи

In [ ]:
# === НАСТРОЙКИ ===
MODEL_NAME = "bert-base-multilingual-cased"  # Модель для русского и английского
# Альтернативы:
# "bert-base-uncased" - английский
# "DeepPavlov/rubert-base-cased" - русский
# "distilbert-base-uncased" - быстрая версия
# "xlm-roberta-base" - многоязычная

TEXT_COLUMN = 'text'       # Название колонки с текстом
LABEL_COLUMN = 'label'     # Название колонки с метками
NUM_LABELS = 2             # Количество классов
MAX_LENGTH = 128           # Максимальная длина текста
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

print(f"Модель: {MODEL_NAME}")
print(f"Количество классов: {NUM_LABELS}")

## 5. Загрузка модели и токенизатора

In [ ]:
# Токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Модель для классификации
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
).to(device)

print("✓ Модель и токенизатор загружены!")

## 6. Подготовка данных

In [ ]:
# Разделение на train/validation
if LABEL_COLUMN in train_df.columns:
    train_data, val_data = train_test_split(
        train_df, 
        test_size=0.1, 
        random_state=42,
        stratify=train_df[LABEL_COLUMN]
    )
else:
    train_data = train_df
    val_data = None

# Преобразование в Dataset
train_dataset = Dataset.from_pandas(train_data)
if val_data is not None:
    val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_df)

print(f"Train size: {len(train_dataset)}")
if val_data is not None:
    print(f"Val size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")

In [ ]:
# Функция токенизации
def tokenize_function(examples):
    return tokenizer(
        examples[TEXT_COLUMN],
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH
    )

# Токенизация датасетов
train_dataset = train_dataset.map(tokenize_function, batched=True)
if val_data is not None:
    val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Удаление ненужных колонок и переименование labels
if LABEL_COLUMN in train_dataset.column_names:
    train_dataset = train_dataset.rename_column(LABEL_COLUMN, "labels")
    if val_data is not None:
        val_dataset = val_dataset.rename_column(LABEL_COLUMN, "labels")

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
if val_data is not None:
    val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask"])

print("✓ Данные токенизированы!")

## 7. Метрики

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1
    }

## 8. Обучение

In [ ]:
# Настройки обучения
training_args = TrainingArguments(
    output_dir="./bert_results",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch" if val_data is not None else "no",
    save_strategy="epoch",
    load_best_model_at_end=True if val_data is not None else False,
    metric_for_best_model="f1" if val_data is not None else None,
    fp16=torch.cuda.is_available(),
)

# Тренер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset if val_data is not None else None,
    compute_metrics=compute_metrics if val_data is not None else None,
)

print("✓ Trainer готов!")

In [ ]:
# Обучение
trainer.train()

# Сохранение
trainer.save_model("./bert_finetuned")
tokenizer.save_pretrained("./bert_finetuned")

print("✓ Модель обучена и сохранена!")

## 9. Оценка на валидации

In [ ]:
if val_data is not None:
    # Оценка
    eval_results = trainer.evaluate()
    print("\nРезультаты на валидации:")
    for key, value in eval_results.items():
        print(f"{key}: {value:.4f}")
    
    # Детальный отчет
    predictions = trainer.predict(val_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    
    print("\nClassification Report:")
    print(classification_report(val_data[LABEL_COLUMN].values, preds))

## 10. Предсказания на тесте

In [ ]:
# Загрузка обученной модели
model_finetuned = AutoModelForSequenceClassification.from_pretrained("./bert_finetuned")
tokenizer_finetuned = AutoTokenizer.from_pretrained("./bert_finetuned")

# Pipeline для предсказаний
classifier = pipeline(
    "text-classification",
    model=model_finetuned,
    tokenizer=tokenizer_finetuned,
    device=0 if torch.cuda.is_available() else -1
)

print("✓ Модель загружена для inference!")

In [ ]:
# Предсказания
test_texts = test_df[TEXT_COLUMN].tolist()

# Предсказание батчами для ускорения
predictions = []
batch_size = 32

for i in range(0, len(test_texts), batch_size):
    batch = test_texts[i:i+batch_size]
    batch_preds = classifier(batch, truncation=True, max_length=MAX_LENGTH)
    predictions.extend([int(pred['label'].split('_')[-1]) for pred in batch_preds])

print(f"✓ Сделано {len(predictions)} предсказаний")

In [ ]:
# === СОЗДАНИЕ SUBMISSION ===
submission = pd.DataFrame({
    'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
    'prediction': predictions
})

submission.to_csv('bert_submission.csv', index=False)
print("✓ Submission сохранен!")
print(submission.head())

## 11. Text Embeddings (извлечение признаков)

In [ ]:
# Использование BERT для получения embeddings
from transformers import AutoModel

# Модель для embeddings (без classification head)
embedding_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
embedding_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def get_embeddings(texts, batch_size=32):
    """Получить embeddings для текстов"""
    embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        # Токенизация
        encoded = embedding_tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors='pt'
        ).to(device)
        
        # Получение embeddings
        with torch.no_grad():
            outputs = embedding_model(**encoded)
            # Используем [CLS] токен (первый токен)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
    
    return np.vstack(embeddings)

# Пример использования
# sample_texts = ["Пример текста 1", "Пример текста 2"]
# embeddings = get_embeddings(sample_texts)
# print(f"Shape embeddings: {embeddings.shape}")

## 12. Inference с разными моделями

In [ ]:
# Быстрый inference с разными pre-trained моделями

# Sentiment Analysis (английский)
# sentiment_pipeline = pipeline("sentiment-analysis")
# result = sentiment_pipeline("I love this movie!")
# print(result)

# Named Entity Recognition
# ner_pipeline = pipeline("ner", grouped_entities=True)
# result = ner_pipeline("Apple was founded by Steve Jobs in California.")
# print(result)

# Question Answering
# qa_pipeline = pipeline("question-answering")
# context = "Python is a programming language."
# question = "What is Python?"
# result = qa_pipeline(question=question, context=context)
# print(result)